# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator
import numpy as np

In [2]:
def MAP(_, row:NamedTuple):
    if (row.gender == 'female'):
        yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
    sum = 0
    count = 0
    for row in rows:
        sum += row.social_contacts
        count += 1
    if (count > 0):
        yield (age, sum/count)
    else:
        yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
    return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str
    
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
    if (row.gender == 'female'):
       yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
    sum = 0
    count = 0
    for row in rows:
        sum += row.social_contacts
        count += 1
    if (count > 0):
        yield (age, sum/count)
    else:
        yield (age, 0)
 
def RECORDREADER():
    return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
    i, j = coordinates
    yield (i, value*vec[j])
 
def REDUCE(i:int, products:Iterator[NamedTuple]):
    sum = 0
    for p in products:
        sum += p
    yield (i, sum)

def RECORDREADER():
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            yield ((i, j), mat[i,j])
      
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.417207055924178),
 (1, 1.417207055924178),
 (2, 1.417207055924178),
 (3, 1.417207055924178),
 (4, 1.417207055924178)]

## Inverted index 

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
    for (docid, document) in enumerate(documents):
        yield ("{}".format(docid), document)
      
def MAP(docId:str, body:str):
    for word in set(body.split(' ')):
       yield (word, docId)
 
def REDUCE(word:str, docIds:Iterator[str]):
    yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('is', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
    for (docid, document) in enumerate(documents):
        for (lineid, line) in enumerate(document.split('\n')):
            yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
    for word in line.split(" "):  
        yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
    sum = 0
    for c in counts:
        sum += c
    yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()
      
def groupbykey_distributed(map_partitions, PARTITIONER):
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for (k2, v2) in map_partition:
            p = partitions[PARTITIONER(k2)]
            p[k2] = p.get(k2, []) + [v2]
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]
 
def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers
  
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
    if COMBINER != None:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  
    print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
    return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
    global maps
  
    def RECORDREADER(split):
        for (docid, document) in enumerate(split):
            for (lineid, line) in enumerate(document.split('\n')):
                yield ("{}:{}".format(docid,lineid), line)
      
    split_size =  int(np.ceil(len(documents)/maps))
    for i in range(0, len(documents), split_size):
        yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
    for word in line.split(" "):  
        yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
    sum = 0
    for c in counts:
        sum += c
    yield (word, sum)
  
# try to set COMBINER=REDUCER and look at the number of values sent over the network 
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('banana', 2), ('is', 18), ('it', 18), ('what', 10)]),
 (1, [('a', 2)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
    global maps
  
    def RECORDREADER(split):
        for value in split:
            yield (value, None)
      
    split_size =  int(np.ceil(len(input_values)/maps))
    for i in range(0, len(input_values), split_size):
        yield RECORDREADER(input_values[i:i+split_size])
    
def MAP(value:int, _):
    yield (value, None)
  
def PARTITIONER(key):
    global reducers
    global max_value
    global min_value
    bucket_size = (max_value-min_value)/reducers
    bucket_id = 0
    while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
        bucket_id += 1
    return bucket_id

def REDUCE(value:int, _):
    yield (None,value)
  
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.055868946742893),
   (None, 0.07176896949085909),
   (None, 0.13631409797518024),
   (None, 0.14046107455920043),
   (None, 0.15461669121097532),
   (None, 0.19481905321649795),
   (None, 0.2574237480179338),
   (None, 0.2747488942942312),
   (None, 0.3596527877710456),
   (None, 0.4510391983526313),
   (None, 0.4691399568982142),
   (None, 0.4691578072642112),
   (None, 0.4728884820521265)]),
 (1,
  [(None, 0.5082693472475383),
   (None, 0.5442857143984235),
   (None, 0.5446894437501602),
   (None, 0.5644964376288164),
   (None, 0.5691281522175504),
   (None, 0.7306899718421087),
   (None, 0.7421973489558943),
   (None, 0.7550311311855304),
   (None, 0.7631707315381367),
   (None, 0.7638074698681306),
   (None, 0.7831751379170145),
   (None, 0.7930112951197739),
   (None, 0.8014465067154266),
   (None, 0.809689181511213),
   (None, 0.927798788603381),
   (None, 0.9847751570830034),
   (None, 0.9894142943106486)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [21]:
import random

n = 150

def MAP(key, value):
    yield (0, value)

def REDUCE(key, row):
    yield max(row)

def RECORDREADER():
    return [(i, random.randint(0, 2 * n)) for i in range(n)]
    
print(*MapReduce(RECORDREADER, MAP, REDUCE))

295


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [22]:
def MAP(key, value):
    yield (1, value)

def REDUCE(key, row):
    yield sum(row) / len(row)

print(*MapReduce(RECORDREADER, MAP, REDUCE))

151.08


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [23]:
n = 20

def groupbykey_sort(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    for k2 in t:
        t[k2] = sorted(t[k2])
    return t.items()

print(groupbykey_sort(flatten(map(lambda x: MAP(*x), RECORDREADER()))))

dict_items([(1, [4, 11, 11, 15, 18, 20, 23, 24, 26, 27, 30, 30, 30, 33, 34, 36, 36, 37, 40, 40])])


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [24]:
n = 150

def MAP(key, value):
    yield (value, value)

def REDUCE(key, rows):
    yield key
    
lst = list(MapReduce(RECORDREADER, MAP, REDUCE))
print(lst)
print(len(lst) != len(set(lst)))

[7, 124, 153, 18, 188, 247, 66, 134, 25, 1, 15, 104, 101, 24, 40, 182, 30, 180, 109, 138, 197, 95, 133, 206, 139, 249, 0, 282, 115, 193, 5, 112, 120, 212, 48, 74, 284, 19, 148, 142, 262, 145, 161, 274, 162, 67, 261, 174, 248, 20, 80, 83, 31, 152, 278, 170, 155, 165, 257, 258, 41, 23, 250, 59, 177, 6, 221, 125, 265, 10, 102, 209, 26, 238, 151, 171, 87, 131, 156, 122, 42, 123, 194, 91, 89, 128, 164, 271, 35, 291, 181, 299, 149, 239, 105, 211, 63, 129, 192, 98, 214, 185, 219, 159, 52, 118, 84, 294, 2, 251, 130, 208, 172, 49, 32, 108]
False


# Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [25]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=22, gender='female', social_contacts=500),
    User(id=3, age=33, gender='male', social_contacts=800),
    User(id=4, age=12, gender='female', social_contacts=1020)
]

def MAP(key, value):
    if value.age > 22:
        yield (value, value)

def REDUCE(key, row):
    yield row

def RECORDREADER():
    return [(user.id, user) for user in input_collection]

res = list(MapReduce(RECORDREADER, MAP, REDUCE))
list(map(lambda x: x[0], sorted(res)))

[User(id=0, age=55, social_contacts=20, gender='male'),
 User(id=1, age=25, social_contacts=240, gender='female'),
 User(id=3, age=33, social_contacts=800, gender='male')]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [26]:
def MAP(key, value):
    yield (value.age, value.age)

def REDUCE(key, rows):
    yield (key, key)

list(MapReduce(RECORDREADER, MAP, REDUCE))

[(55, 55), (25, 25), (22, 22), (33, 33), (12, 12)]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [27]:
input_collection_1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=2, age=25, gender='male', social_contacts=240),
    User(id=4, age=17, gender='female', social_contacts=500),
    User(id=5, age=40, gender='male', social_contacts=800)
]
input_collection_2 = [
    User(id=2, age=25, gender='male', social_contacts=240),
    User(id=3, age=17, gender='female', social_contacts=500),
    User(id=5, age=40, gender='male', social_contacts=800),
]

def MAP(key, value):
    yield (value, value)

def REDUCE(key, row):
    yield (key, key)

def RECORDREADER():
    return [(user.id, user) for user in input_collection_1 + input_collection_2]

list(MapReduce(RECORDREADER, MAP, REDUCE))


[(User(id=0, age=55, social_contacts=20, gender='male'),
  User(id=0, age=55, social_contacts=20, gender='male')),
 (User(id=2, age=25, social_contacts=240, gender='male'),
  User(id=2, age=25, social_contacts=240, gender='male')),
 (User(id=4, age=17, social_contacts=500, gender='female'),
  User(id=4, age=17, social_contacts=500, gender='female')),
 (User(id=5, age=40, social_contacts=800, gender='male'),
  User(id=5, age=40, social_contacts=800, gender='male')),
 (User(id=3, age=17, social_contacts=500, gender='female'),
  User(id=3, age=17, social_contacts=500, gender='female'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [28]:
def REDUCE(key, rows):
    if len(rows) > 1:
        yield (key, key)

list(MapReduce(RECORDREADER, MAP, REDUCE))

[(User(id=2, age=25, social_contacts=240, gender='male'),
  User(id=2, age=25, social_contacts=240, gender='male')),
 (User(id=5, age=40, social_contacts=800, gender='male'),
  User(id=5, age=40, social_contacts=800, gender='male'))]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [29]:
def MAP(key, value):
    if value in input_collection_1:
        yield (value, 0)
    elif value in input_collection_2:
        yield (value, 1)

def REDUCE(key, rows):
    if (rows == [0]):
        yield (key, key)

list(MapReduce(RECORDREADER, MAP, REDUCE))

[(User(id=0, age=55, social_contacts=20, gender='male'),
  User(id=0, age=55, social_contacts=20, gender='male')),
 (User(id=4, age=17, social_contacts=500, gender='female'),
  User(id=4, age=17, social_contacts=500, gender='female'))]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [30]:
class Student(NamedTuple):
    Name: str
    Age: int
    TeacherName: str

class StudentInfo(NamedTuple):
    Name: str
    Age: int
    
class Teacher(NamedTuple):
    TeacherName: str
    Experience: int

class Teacher_Experience(NamedTuple):
    Experience: int

class Student_Teacher(NamedTuple):
    Name: str
    Age: int
    TeacherName: str
    Experience: int

students_collection = [
    Student(Name="Mark", Age=17, TeacherName="Samara"),
    Student(Name="Andrey", Age=16, TeacherName="Petr"),
    Student(Name="Zoi", Age=17, TeacherName="Samara"),
    Student(Name="Dasha", Age=18, TeacherName="Petr"),
    Student(Name="Nikita", Age=19, TeacherName="Oksana"),
    Student(Name="Maksim", Age=16, TeacherName="Petr"),
    Student(Name="Lena", Age=15, TeacherName="Samara")
]
teachers_collection = [
    Teacher(TeacherName="Samara", Experience=7),
    Teacher(TeacherName="Oksana", Experience=5),
    Teacher(TeacherName="Petr", Experience=25),
    Teacher(TeacherName="Alexey", Experience=9)
]

def MAP(key, value):
    if value in students_collection:
        yield (value.TeacherName, (0, StudentInfo(value.Name, value.Age)))
    elif value in teachers_collection:
        yield (value.TeacherName, (1, Teacher_Experience(value.Experience)))

def REDUCE(key, rows):
    for i in range(len(rows)):
        for j in range(len(rows)):
            if (rows[i][0] == 1 and rows[j][0] == 0):
                yield (key, Student_Teacher(Name=rows[j][1].Name, Age=rows[j][1].Age, TeacherName=key, Experience=rows[i][1].Experience))

def RECORDREADER():
    return [(idx, obj) for idx, obj in enumerate(students_collection + teachers_collection)]

res = list(MapReduce(RECORDREADER, MAP, REDUCE))

teachers = { teacher[0]:[] for teacher in res }

for student in res:
    teachers[student[1].TeacherName].append(student[1])

for teacherName in teachers:
    print(teacherName)
    for student in teachers[teacherName]:
        print("  ", student)

Samara
   Student_Teacher(Name='Mark', Age=17, TeacherName='Samara', Experience=7)
   Student_Teacher(Name='Zoi', Age=17, TeacherName='Samara', Experience=7)
   Student_Teacher(Name='Lena', Age=15, TeacherName='Samara', Experience=7)
Petr
   Student_Teacher(Name='Andrey', Age=16, TeacherName='Petr', Experience=25)
   Student_Teacher(Name='Dasha', Age=18, TeacherName='Petr', Experience=25)
   Student_Teacher(Name='Maksim', Age=16, TeacherName='Petr', Experience=25)
Oksana
   Student_Teacher(Name='Nikita', Age=19, TeacherName='Oksana', Experience=5)


### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [31]:
class Employee(NamedTuple):
    id: int
    position: str
    salary: int 
    
input_collection = [
    Employee(id=1, position="Director", salary=150),
    Employee(id=2, position="Developer", salary=50),
    Employee(id=3, position="Сleaner", salary=25),
    Employee(id=4, position="Director", salary=250),
    Employee(id=5, position="Developer", salary=70),
    Employee(id=6, position="Director", salary=200),
    Employee(id=7, position="Сleaner", salary=45),
    Employee(id=8, position="Developer", salary=60),
    Employee(id=9, position="Developer", salary=55),
]

def MAP(key, value):
    yield (value.position, value.salary)

def REDUCE(key, rows):
    yield (key, sum(rows))

def RECORDREADER():
    return [(idx, obj) for idx, obj in enumerate(input_collection)]

print("Зарплаты сотрудников в зависимости от их должности:", *MapReduce(RECORDREADER, MAP, REDUCE))

Зарплаты сотрудников в зависимости от их должности: ('Director', 600) ('Developer', 235) ('Сleaner', 70)


### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [32]:
rows, cols = 7, 6
sections = 3

num_cols = cols // sections

matrix = [np.ones((rows, num_cols)) for i in range(sections)]
vector= [np.random.rand(num_cols) for i in range(sections)]

def MAP(coords, value):
  i, j = coords
  yield (i, value * vec[j])

def REDUCE(i, products):
    sum = 0
    for p in products:
        sum += p
    yield (i, sum)

def RECORDREADER():
    global vec
    for idx, section in enumerate(matrix):
        vec = vector[idx]
        for i in range(section.shape[0]):
            for j in range(section.shape[1]):
                yield ((i, j), section[i, j])

print("matrix:", *matrix)
print("vector:", *vector)
list(MapReduce(RECORDREADER, MAP, REDUCE))

matrix: [[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]] [[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]] [[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
vector: [0.17947297 0.76663042] [0.06387788 0.15162044] [0.44403511 0.09919035]


[(0, 1.7048271685686323),
 (1, 1.7048271685686323),
 (2, 1.7048271685686323),
 (3, 1.7048271685686323),
 (4, 1.7048271685686323),
 (5, 1.7048271685686323),
 (6, 1.7048271685686323)]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$. 





In [33]:
# MapReduce model
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [34]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield ((j,k), big_mat[j,k])
      
def MAP(k1, v1):
    (j, k) = k1
    w = v1
    for i in range(I):
        yield ((i, k), small_mat[i, j] * w)
  
def REDUCE(key, values):
    (i, k) = key
    yield ((i, k), sum(values))

Проверьте своё решение

In [35]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i,k), vw) in reduce_output)+1
    K = max(k for ((i,k), vw) in reduce_output)+1
    mat = np.empty(shape=(I,K))
    for ((i,k), vw) in reduce_output:
        mat[i,k] = vw
    return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [36]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [37]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
    for i in range(I):
        for j in range(J):
          yield ((0, i, j), small_mat[i, j])

    for j in range(J):
        for k in range(K):
            yield ((1, j, k), big_mat[j, k])
      
def MAP(k1, v1):
    (matrix_index, row_index, col_index) = k1
    if matrix_index == 0:
        yield (col_index, (matrix_index, row_index, v1))
    else:
        yield (row_index, (matrix_index, col_index, v1))


def REDUCE(key, values):
    matrix_1 = {value[1]: value[2] for value in values if value[0] == 0}
    matrix_2 = {value[1]: value[2] for value in values if value[0] == 1}

    for idx1, val1 in matrix_1.items():
        for idx2, val2 in matrix_2.items():
            yield ((idx1, idx2), val1 * val2)

def MAP_IDENTITY(*args):
    yield args

def REDUCE_AGREGATE(key, values):
    yield (key, sum(values))

def RECORDREADER_IDENTITY(iterable):
    def func():
        for el in iterable:
            yield el
    return func


# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
partial_solution = MapReduce(RECORDREADER, MAP, REDUCE)
solution = MapReduce(RECORDREADER_IDENTITY(partial_solution), MAP_IDENTITY, REDUCE_AGREGATE)

def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i,k), vw) in reduce_output)+1
    K = max(k for ((i,k), vw) in reduce_output)+1
    mat = np.empty(shape=(I,K))
    for ((i,k), vw) in reduce_output:
        mat[i,k] = vw
    return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER. 

In [38]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

maps = 2
reducers = 2

def INPUTFORMAT():
    def RECORDREADER(index, matrix):
        rows, cols = matrix.shape
        for i in range(rows):
            for j in range(cols):
                yield ((index, i, j), matrix[i, j])

    yield RECORDREADER(0, small_mat)
    yield RECORDREADER(1, big_mat)

def MAP(k1, v1):
    (matrix_index, row_index, col_index) = k1
    if matrix_index == 0:
        yield (col_index, (matrix_index, row_index, v1))
    else:
        yield (row_index, (matrix_index, col_index, v1))


def REDUCE(key, values):
    matrix_1 = {value[1]: value[2] for value in values if value[0] == 0}
    matrix_2 = {value[1]: value[2] for value in values if value[0] == 1}

    for idx1, val1 in matrix_1.items():
        for idx2, val2 in matrix_2.items():
            yield ((idx1, idx2), val1 * val2)

def MAP_IDENTITY(*args):
    yield args

def REDUCE_AGREGATE(key, values):
    yield (key, sum(values))

def GET_INPUTFORMAT(iterable):
    def func():
        for el in iterable:
            yield el[1]
    return func

reference_solution = np.matmul(small_mat, big_mat)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

solution = MapReduceDistributed(GET_INPUTFORMAT(partitioned_output), MAP_IDENTITY, REDUCE_AGREGATE, COMBINER=None)
solution = [(partition_id, list(partition)) for (partition_id, partition) in solution]

solution_unpacked = []
for value in solution:
    for v in value[1]:
        solution_unpacked.append(v)
def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i,k), vw) in reduce_output)+1
    K = max(k for ((i,k), vw) in reduce_output)+1
    mat = np.empty(shape=(I,K))
    for ((i,k), vw) in reduce_output:
        mat[i,k] = vw
    return mat

np.allclose(reference_solution, asmatrix(solution_unpacked)) # should return true

126 key-value pairs were sent over a network.
240 key-value pairs were sent over a network.


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [39]:
I = 2
J = 3
K = 4
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

maps = 2
reducers = 2

def INPUTFORMAT():
    def RECORDREADER(index, matrix, shift):
        rows, cols = matrix.shape
        for i in range(rows):
            for j in range(cols):
                yield ((index, i + shift, j), matrix[i, j])

    split_size = int(np.ceil(len(small_mat) / maps))
    for i in range(0, len(small_mat), split_size):
        yield RECORDREADER(0, small_mat[i:i + split_size], i)
    
    split_size = int(np.ceil(len(big_mat) / maps))    
    for i in range(0, len(big_mat), split_size):
        yield RECORDREADER(1, big_mat[i:i + split_size], i)

def MAP(k1, v1):
    (matrix_index, row_index, col_index) = k1
    if matrix_index == 0:
        yield (col_index, (matrix_index, row_index, v1))
    else:
        yield (row_index, (matrix_index, col_index, v1))


def REDUCE(key, values):
    matrix_1 = {value[1]: value[2] for value in values if value[0] == 0}
    matrix_2 = {value[1]: value[2] for value in values if value[0] == 1}

    for idx1, val1 in matrix_1.items():
        for idx2, val2 in matrix_2.items():
            yield ((idx1, idx2), val1 * val2)

def MAP_IDENTITY(*args):
    yield args

def REDUCE_AGREGATE(key, values):
    yield (key, sum(values))

def GET_INPUTFORMAT(iterable):
    def func():
        for el in iterable:
            yield el[1]
    return func

reference_solution = np.matmul(small_mat, big_mat)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

solution = MapReduceDistributed(GET_INPUTFORMAT(partitioned_output), MAP_IDENTITY, REDUCE_AGREGATE, COMBINER=None)
solution = [(partition_id, list(partition)) for (partition_id, partition) in solution]

solution_unpacked = []
for value in solution:
    for v in value[1]:
        solution_unpacked.append(v)
def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i,k), vw) in reduce_output)+1
    K = max(k for ((i,k), vw) in reduce_output)+1
    mat = np.empty(shape=(I,K))
    for ((i,k), vw) in reduce_output:
        mat[i,k] = vw
    return mat

np.allclose(reference_solution, asmatrix(solution_unpacked)) # should return true

18 key-value pairs were sent over a network.
24 key-value pairs were sent over a network.


True